In [2]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier
#!pip install catboost
import catboost as cat
from catboost import CatBoostClassifier, CatBoostClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam

In [3]:
def one_hot_encoding(traindata, testdata, *args):
    
    for ii in args:
        traindata = pd.get_dummies(traindata, prefix=[ii], columns=[ii])
        testdata = pd.get_dummies(testdata, prefix=[ii], columns=[ii])
        
    return traindata, testdata

def drop_columns(traindata, testdata, target, *args):
    
    labels = np.array(traindata[target])
    
    columns = []
    for _ in args:
        columns.append(_)
        
    traindata = traindata.drop(columns, axis=1)
    #traindata = traindata.drop(target, axis=1)
    testdata = testdata.drop(columns, axis=1)
        
    return labels, traindata, testdata

In [4]:
def process(traindata, testdata):
    
    cols = list(traindata.columns)
    for _ in cols:
        traindata[_] = np.where(traindata[_] == np.inf, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == np.inf, 0, testdata[_])
        traindata[_] = np.where(traindata[_] == np.nan, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == np.nan, 0, testdata[_])
        traindata[_] = np.where(traindata[_] == -np.inf, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == -np.inf, 0, testdata[_])
        
    return traindata, testdata

In [5]:
def freq_encode(data, cols):
    for i in cols:
        encoding = data.groupby(i).size()
        encoding = encoding/len(data)
        data[i + '_enc'] = data[i].map(encoding)
    return data

def mean_target(data, cols):
    kf = KFold(5)
    a = pd.DataFrame()
    for tr_ind, val_ind in kf.split(data):
        X_tr, X_val= data.iloc[tr_ind].copy(), data.iloc[val_ind].copy()
        for col in cols:
            means = X_val[col].map(X_tr.groupby(col).CHURN.mean())
            X_val[col + '_mean_target'] = means + 0.0001
        a = pd.concat((a, X_val))
    prior = target.mean()
    a.fillna(prior, inplace=True)
    return a

In [6]:
#importing files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('sample_submission.csv')

In [7]:
ntrain = train.shape[0]
ntest = test.shape[0]
target = train.CHURN.copy()
df = pd.concat((train, test)).reset_index(drop=True)

In [8]:
cols = ['ZONE1', 'ZONE2', 'TIGO', 'user_id', 'MRG']

labels, train1, test1 = drop_columns(df, test, 'CHURN', *cols)

one_hot_cols = ['REGION', 'TENURE']

#df, test1 = one_hot_encoding(df, test, *one_hot_cols)

In [9]:
cols = ['ZONE1', 'ZONE2', 'TIGO', 'user_id', 'MRG']

labels, df, test1 = drop_columns(df, test, 'CHURN', *cols)

In [10]:
df.shape, test1.shape

((500000, 14), (100000, 13))

In [11]:
df = freq_encode(df, ['TOP_PACK', 'REGION', 'TENURE'])
data = df.copy()

In [18]:
data.shape

(500000, 40)

In [13]:
data = freq_encode(data, ['REGION', 'TENURE'])
data = mean_target(data, ['REGION', 'TOP_PACK', 'TENURE'])

In [15]:
data = data.drop('TOP_PACK', axis=1)

In [17]:
data, test1 = one_hot_encoding(data, test, *one_hot_cols)

In [19]:
train2 = data[:ntrain].copy()
target = train2.CHURN.copy()
train2.drop('CHURN', axis=1, inplace=True)

test2 = data[ntrain:].copy()
test2.drop('CHURN', axis=1, inplace=True)
test2 = test2.reset_index(drop=True)

In [20]:
traindata = train2.copy()
testdata = test2.copy()

In [21]:
traindata['REG*REV'] = traindata['REGULARITY'] * traindata['REVENUE']
traindata['REG/REV'] = traindata['REGULARITY'] / traindata['REVENUE']
traindata['LOG_REV'] = np.log(traindata['REVENUE'])
traindata['REG*MUT'] = traindata['REGULARITY'] * traindata['MONTANT']
traindata['REG/MUT'] = traindata['REGULARITY'] / traindata['MONTANT']
traindata['LOG_MUT'] = np.log(traindata['MONTANT'])
traindata['REG*NET'] = traindata['REGULARITY'] * traindata['ON_NET']
traindata['REG/NET'] = traindata['REGULARITY'] / traindata['ON_NET']
traindata['FREQ+FREQ'] = traindata['FREQUENCE_RECH'] + traindata['FREQUENCE']
traindata['FREQ-FREQ'] = traindata['FREQUENCE_RECH'] - traindata['FREQUENCE']
traindata['LOG_NET'] = np.log(traindata['ON_NET'])
traindata['REV+DAT'] = traindata['REVENUE'] + traindata['DATA_VOLUME']
traindata['REV-DAT'] = traindata['REVENUE'] - traindata['DATA_VOLUME']
traindata['REV-DAT'] = traindata['REVENUE'] - traindata['DATA_VOLUME']
traindata['REV/DAT'] = traindata['REVENUE'] / traindata['DATA_VOLUME']
traindata['REV-ARP'] = traindata['REVENUE'] - traindata['ARPU_SEGMENT']
traindata['REV+ARP'] = traindata['REVENUE'] + traindata['ARPU_SEGMENT']
traindata['REV*ARP'] = traindata['REVENUE'] * traindata['ARPU_SEGMENT']
traindata['REV/ARP'] = traindata['REVENUE'] / traindata['ARPU_SEGMENT']
traindata['REV*FREQ'] = traindata['REVENUE'] * traindata['FREQUENCE']
traindata['REV/FREQ'] = traindata['REVENUE'] / traindata['FREQUENCE']
traindata['LOG_DAT'] = np.log(traindata['DATA_VOLUME'])
traindata['LOG_ARP'] = np.log(traindata['ARPU_SEGMENT'])

/home/olawale/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
testdata['REG*REV'] = testdata['REGULARITY'] * testdata['REVENUE']
testdata['REG/REV'] = testdata['REGULARITY'] / testdata['REVENUE']
testdata['LOG_REV'] = np.log(testdata['REVENUE'])
testdata['REG*MUT'] = testdata['REGULARITY'] * testdata['MONTANT']
testdata['REG/MUT'] = testdata['REGULARITY'] / testdata['MONTANT']
testdata['LOG_MUT'] = np.log(testdata['MONTANT'])
testdata['REG*NET'] = testdata['REGULARITY'] * testdata['ON_NET']
testdata['REG/NET'] = testdata['REGULARITY'] / testdata['ON_NET']
testdata['FREQ+FREQ'] = testdata['FREQUENCE_RECH'] + testdata['FREQUENCE']
testdata['FREQ-FREQ'] = testdata['FREQUENCE_RECH'] - testdata['FREQUENCE']
testdata['LOG_NET'] = np.log(testdata['ON_NET'])
testdata['REV+DAT'] = testdata['REVENUE'] + testdata['DATA_VOLUME']
testdata['REV-DAT'] = testdata['REVENUE'] - testdata['DATA_VOLUME']
testdata['REV-DAT'] = testdata['REVENUE'] - testdata['DATA_VOLUME']
testdata['REV/DAT'] = testdata['REVENUE'] / testdata['DATA_VOLUME']
testdata['REV-ARP'] = testdata['REVENUE'] - testdata['ARPU_SEGMENT']
testdata['REV+ARP'] = testdata['REVENUE'] + testdata['ARPU_SEGMENT']
testdata['REV*ARP'] = testdata['REVENUE'] * testdata['ARPU_SEGMENT']
testdata['REV/ARP'] = testdata['REVENUE'] / testdata['ARPU_SEGMENT']
testdata['REV*FREQ'] = testdata['REVENUE'] * testdata['FREQUENCE']
testdata['REV/FREQ'] = testdata['REVENUE'] / testdata['FREQUENCE']
testdata['LOG_DAT'] = np.log(testdata['DATA_VOLUME'])
testdata['LOG_ARP'] = np.log(testdata['ARPU_SEGMENT'])

In [23]:
traindata, testdata = process(traindata, testdata)

In [47]:
traindata.shape, testdata.shape

((400000, 61), (100000, 61))

In [25]:
traindata = traindata.fillna(-999)
testdata = testdata.fillna(-999)

In [26]:
#using StandardScaler function to scale the numeric features 

scaler = preprocessing.StandardScaler().fit(traindata)
traindata = scaler.transform(traindata)
traindata

array([[ 2.12962213,  2.04161944,  2.24281355, ...,  0.51645367,
        -0.77327583,  1.161757  ],
       [ 0.11332956,  1.79126678,  0.12113158, ..., -0.31690067,
         1.44757293,  0.82403724],
       [-0.33120738, -0.37845629, -0.33640641, ...,  0.45663759,
        -0.77327583,  0.56336291],
       ...,
       [-0.45821794, -0.46190718, -0.46161581, ..., -0.27591793,
         0.96044918,  0.37948656],
       [-0.56932247, -0.61319446, -0.5694741 , ..., -0.54193198,
        -0.36653221, -0.66862769],
       [-0.56932247, -0.61319446, -0.5708517 , ..., -0.55820195,
        -0.77327583, -1.33680034]])

In [27]:
testdata = scaler.transform(testdata)
testdata

array([[-0.56932247, -0.61319446, -0.5708517 , ..., -0.55820195,
        -0.77327583, -1.33680034],
       [-0.56932247, -0.61319446, -0.56931779, ..., -0.53989823,
        -0.77327583, -0.66862769],
       [-0.26770211,  0.62295436, -0.21150964, ..., -0.33758887,
         1.09581674,  0.66608685],
       ...,
       [-0.28357843, -0.04465274, -0.28966782, ..., -0.15371235,
         0.69990672,  0.60726804],
       [-0.17244419, -0.2950054 , -0.1777453 , ...,  0.71848246,
        -0.77327583,  0.68772038],
       [-0.56932247, -0.61319446, -0.5708517 , ..., -0.55820195,
        -0.77327583, -1.33680034]])

In [28]:
x_train, x_test, y_train, y_test = ms.train_test_split(traindata, train.CHURN, test_size=0.2,
                                                       random_state=42)

In [33]:
lg = lgb.LGBMClassifier()

params = {'n_estimators': [300],
          'max_depth': [9, 10, 11, 12]}

eval_set = [(x_train, y_train), (x_test, y_test)]
clf = GridSearchCV(lg, params)
clf.fit(x_train, y_train, eval_metric="logloss", eval_set=eval_set, verbose=2, early_stopping_rounds = 30)

Training until validation scores don't improve for 30 rounds
[2]	valid_0's binary_logloss: 0.407062	valid_1's binary_logloss: 0.407862
[4]	valid_0's binary_logloss: 0.363363	valid_1's binary_logloss: 0.36465
[6]	valid_0's binary_logloss: 0.334052	valid_1's binary_logloss: 0.335713
[8]	valid_0's binary_logloss: 0.313315	valid_1's binary_logloss: 0.31532
[10]	valid_0's binary_logloss: 0.298218	valid_1's binary_logloss: 0.300533
[12]	valid_0's binary_logloss: 0.287052	valid_1's binary_logloss: 0.289648
[14]	valid_0's binary_logloss: 0.278638	valid_1's binary_logloss: 0.281453
[16]	valid_0's binary_logloss: 0.272243	valid_1's binary_logloss: 0.275271
[18]	valid_0's binary_logloss: 0.267369	valid_1's binary_logloss: 0.270609
[20]	valid_0's binary_logloss: 0.263608	valid_1's binary_logloss: 0.267058
[22]	valid_0's binary_logloss: 0.260667	valid_1's binary_logloss: 0.264355
[24]	valid_0's binary_logloss: 0.258363	valid_1's binary_logloss: 0.262265
[26]	valid_0's binary_logloss: 0.256549	valid

[38]	valid_0's binary_logloss: 0.251057	valid_1's binary_logloss: 0.256358
[40]	valid_0's binary_logloss: 0.250604	valid_1's binary_logloss: 0.256133
[42]	valid_0's binary_logloss: 0.250185	valid_1's binary_logloss: 0.255934
[44]	valid_0's binary_logloss: 0.249835	valid_1's binary_logloss: 0.255771
[46]	valid_0's binary_logloss: 0.249545	valid_1's binary_logloss: 0.255686
[48]	valid_0's binary_logloss: 0.249268	valid_1's binary_logloss: 0.255549
[50]	valid_0's binary_logloss: 0.249042	valid_1's binary_logloss: 0.255479
[52]	valid_0's binary_logloss: 0.248826	valid_1's binary_logloss: 0.255431
[54]	valid_0's binary_logloss: 0.248625	valid_1's binary_logloss: 0.255397
[56]	valid_0's binary_logloss: 0.248459	valid_1's binary_logloss: 0.255396
[58]	valid_0's binary_logloss: 0.248259	valid_1's binary_logloss: 0.255378
[60]	valid_0's binary_logloss: 0.248044	valid_1's binary_logloss: 0.255327
[62]	valid_0's binary_logloss: 0.247869	valid_1's binary_logloss: 0.255323
[64]	valid_0's binary_log

[70]	valid_0's binary_logloss: 0.247346	valid_1's binary_logloss: 0.255485
[72]	valid_0's binary_logloss: 0.247185	valid_1's binary_logloss: 0.255514
[74]	valid_0's binary_logloss: 0.24704	valid_1's binary_logloss: 0.255521
[76]	valid_0's binary_logloss: 0.246864	valid_1's binary_logloss: 0.25552
[78]	valid_0's binary_logloss: 0.246754	valid_1's binary_logloss: 0.255544
[80]	valid_0's binary_logloss: 0.246618	valid_1's binary_logloss: 0.255559
[82]	valid_0's binary_logloss: 0.246473	valid_1's binary_logloss: 0.255612
[84]	valid_0's binary_logloss: 0.246351	valid_1's binary_logloss: 0.25561
[86]	valid_0's binary_logloss: 0.246223	valid_1's binary_logloss: 0.255605
Early stopping, best iteration is:
[57]	valid_0's binary_logloss: 0.248357	valid_1's binary_logloss: 0.25542
Training until validation scores don't improve for 30 rounds
[2]	valid_0's binary_logloss: 0.407062	valid_1's binary_logloss: 0.407862
[4]	valid_0's binary_logloss: 0.363363	valid_1's binary_logloss: 0.36465
[6]	valid_0

[16]	valid_0's binary_logloss: 0.272298	valid_1's binary_logloss: 0.275373
[18]	valid_0's binary_logloss: 0.267391	valid_1's binary_logloss: 0.270724
[20]	valid_0's binary_logloss: 0.263626	valid_1's binary_logloss: 0.267147
[22]	valid_0's binary_logloss: 0.260711	valid_1's binary_logloss: 0.264432
[24]	valid_0's binary_logloss: 0.258413	valid_1's binary_logloss: 0.262324
[26]	valid_0's binary_logloss: 0.256568	valid_1's binary_logloss: 0.260679
[28]	valid_0's binary_logloss: 0.255128	valid_1's binary_logloss: 0.259405
[30]	valid_0's binary_logloss: 0.253966	valid_1's binary_logloss: 0.25843
[32]	valid_0's binary_logloss: 0.253016	valid_1's binary_logloss: 0.257683
[34]	valid_0's binary_logloss: 0.252254	valid_1's binary_logloss: 0.257128
[36]	valid_0's binary_logloss: 0.251609	valid_1's binary_logloss: 0.256712
[38]	valid_0's binary_logloss: 0.251077	valid_1's binary_logloss: 0.256365
[40]	valid_0's binary_logloss: 0.250645	valid_1's binary_logloss: 0.256104
[42]	valid_0's binary_logl

[38]	valid_0's binary_logloss: 0.251069	valid_1's binary_logloss: 0.2565
[40]	valid_0's binary_logloss: 0.250619	valid_1's binary_logloss: 0.25623
[42]	valid_0's binary_logloss: 0.250222	valid_1's binary_logloss: 0.256024
[44]	valid_0's binary_logloss: 0.249898	valid_1's binary_logloss: 0.255842
[46]	valid_0's binary_logloss: 0.249583	valid_1's binary_logloss: 0.255694
[48]	valid_0's binary_logloss: 0.249286	valid_1's binary_logloss: 0.255605
[50]	valid_0's binary_logloss: 0.249048	valid_1's binary_logloss: 0.255527
[52]	valid_0's binary_logloss: 0.24884	valid_1's binary_logloss: 0.255476
[54]	valid_0's binary_logloss: 0.248598	valid_1's binary_logloss: 0.255451
[56]	valid_0's binary_logloss: 0.248391	valid_1's binary_logloss: 0.25544
[58]	valid_0's binary_logloss: 0.248197	valid_1's binary_logloss: 0.255402
[60]	valid_0's binary_logloss: 0.248018	valid_1's binary_logloss: 0.255435
[62]	valid_0's binary_logloss: 0.247846	valid_1's binary_logloss: 0.255416
[64]	valid_0's binary_logloss:

[54]	valid_0's binary_logloss: 0.248553	valid_1's binary_logloss: 0.255515
[56]	valid_0's binary_logloss: 0.248337	valid_1's binary_logloss: 0.255462
[58]	valid_0's binary_logloss: 0.248132	valid_1's binary_logloss: 0.255461
[60]	valid_0's binary_logloss: 0.247961	valid_1's binary_logloss: 0.255451
[62]	valid_0's binary_logloss: 0.24779	valid_1's binary_logloss: 0.255452
[64]	valid_0's binary_logloss: 0.247628	valid_1's binary_logloss: 0.255459
[66]	valid_0's binary_logloss: 0.247477	valid_1's binary_logloss: 0.255471
[68]	valid_0's binary_logloss: 0.247329	valid_1's binary_logloss: 0.255464
[70]	valid_0's binary_logloss: 0.247164	valid_1's binary_logloss: 0.255463
[72]	valid_0's binary_logloss: 0.247049	valid_1's binary_logloss: 0.255481
[74]	valid_0's binary_logloss: 0.24688	valid_1's binary_logloss: 0.25548
[76]	valid_0's binary_logloss: 0.24671	valid_1's binary_logloss: 0.2555
[78]	valid_0's binary_logloss: 0.246573	valid_1's binary_logloss: 0.255526
[80]	valid_0's binary_logloss: 

[84]	valid_0's binary_logloss: 0.246387	valid_1's binary_logloss: 0.255915
[86]	valid_0's binary_logloss: 0.246254	valid_1's binary_logloss: 0.255875
Early stopping, best iteration is:
[57]	valid_0's binary_logloss: 0.248319	valid_1's binary_logloss: 0.255489
Training until validation scores don't improve for 30 rounds
[2]	valid_0's binary_logloss: 0.407038	valid_1's binary_logloss: 0.407881
[4]	valid_0's binary_logloss: 0.363302	valid_1's binary_logloss: 0.364628
[6]	valid_0's binary_logloss: 0.333985	valid_1's binary_logloss: 0.33572
[8]	valid_0's binary_logloss: 0.313271	valid_1's binary_logloss: 0.315331
[10]	valid_0's binary_logloss: 0.298177	valid_1's binary_logloss: 0.300537
[12]	valid_0's binary_logloss: 0.286994	valid_1's binary_logloss: 0.28964
[14]	valid_0's binary_logloss: 0.278587	valid_1's binary_logloss: 0.281489
[16]	valid_0's binary_logloss: 0.272201	valid_1's binary_logloss: 0.275326
[18]	valid_0's binary_logloss: 0.267322	valid_1's binary_logloss: 0.270681
[20]	valid

[16]	valid_0's binary_logloss: 0.272253	valid_1's binary_logloss: 0.275427
[18]	valid_0's binary_logloss: 0.267346	valid_1's binary_logloss: 0.27076
[20]	valid_0's binary_logloss: 0.26357	valid_1's binary_logloss: 0.267177
[22]	valid_0's binary_logloss: 0.260626	valid_1's binary_logloss: 0.264444
[24]	valid_0's binary_logloss: 0.258326	valid_1's binary_logloss: 0.262349
[26]	valid_0's binary_logloss: 0.256503	valid_1's binary_logloss: 0.260739
[28]	valid_0's binary_logloss: 0.255077	valid_1's binary_logloss: 0.259514
[30]	valid_0's binary_logloss: 0.253922	valid_1's binary_logloss: 0.258545
[32]	valid_0's binary_logloss: 0.253001	valid_1's binary_logloss: 0.257794
[34]	valid_0's binary_logloss: 0.252219	valid_1's binary_logloss: 0.257229
[36]	valid_0's binary_logloss: 0.251574	valid_1's binary_logloss: 0.256835
[38]	valid_0's binary_logloss: 0.251049	valid_1's binary_logloss: 0.256487
[40]	valid_0's binary_logloss: 0.250588	valid_1's binary_logloss: 0.256218
[42]	valid_0's binary_loglo

[56]	valid_0's binary_logloss: 0.248382	valid_1's binary_logloss: 0.255621
[58]	valid_0's binary_logloss: 0.248264	valid_1's binary_logloss: 0.255685
[60]	valid_0's binary_logloss: 0.248041	valid_1's binary_logloss: 0.255666
[62]	valid_0's binary_logloss: 0.247846	valid_1's binary_logloss: 0.255659
[64]	valid_0's binary_logloss: 0.247675	valid_1's binary_logloss: 0.255667
[66]	valid_0's binary_logloss: 0.247505	valid_1's binary_logloss: 0.25567
[68]	valid_0's binary_logloss: 0.247342	valid_1's binary_logloss: 0.25569
[70]	valid_0's binary_logloss: 0.247185	valid_1's binary_logloss: 0.255686
[72]	valid_0's binary_logloss: 0.24701	valid_1's binary_logloss: 0.255686
[74]	valid_0's binary_logloss: 0.246973	valid_1's binary_logloss: 0.255841
[76]	valid_0's binary_logloss: 0.246927	valid_1's binary_logloss: 0.255877
[78]	valid_0's binary_logloss: 0.24677	valid_1's binary_logloss: 0.255828
[80]	valid_0's binary_logloss: 0.246633	valid_1's binary_logloss: 0.255802
[82]	valid_0's binary_logloss

GridSearchCV(estimator=LGBMClassifier(),
             param_grid={'max_depth': [9, 10, 11, 12], 'n_estimators': [300]})

In [31]:
clf.best_params_

{'max_depth': 10, 'n_estimators': 300}

In [32]:
clf.best_estimator_

LGBMClassifier(max_depth=10, n_estimators=300)

In [34]:
clf.best_params_

{'max_depth': 10, 'n_estimators': 300}

In [43]:
kf = KFold(n_splits=5,shuffle=False)
lgbm = lgb.LGBMClassifier(n_estimators=50000, random_state=42,max_depth=10,learning_rate=0.0333, use_best_model=True, eval_metric='Logloss')
scores = []
pred_test = np.zeros((len(test1),2))
for (train_index,test_index) in kf.split(pd.DataFrame(traindata),target):
    X_train,X_test = pd.DataFrame(traindata).iloc[train_index], pd.DataFrame(traindata).iloc[test_index]
    y_train,y_test = train.CHURN.iloc[train_index],train.CHURN.iloc[test_index]
    lgbm.fit(X_train,y_train,early_stopping_rounds=200,eval_set=[(X_test,y_test)])
    #scores.append(metric(lgbm.predict_proba(X_test),y_test))
    pred_test+=lgbm.predict_proba(testdata)

[1]	valid_0's binary_logloss: 0.467008
Training until validation scores don't improve for 200 rounds
[2]	valid_0's binary_logloss: 0.453588
[3]	valid_0's binary_logloss: 0.441461
[4]	valid_0's binary_logloss: 0.430409
[5]	valid_0's binary_logloss: 0.420267
[6]	valid_0's binary_logloss: 0.410918
[7]	valid_0's binary_logloss: 0.402267
[8]	valid_0's binary_logloss: 0.394249
[9]	valid_0's binary_logloss: 0.386776
[10]	valid_0's binary_logloss: 0.379809
[11]	valid_0's binary_logloss: 0.373292
[12]	valid_0's binary_logloss: 0.367188
[13]	valid_0's binary_logloss: 0.361454
[14]	valid_0's binary_logloss: 0.356064
[15]	valid_0's binary_logloss: 0.350993
[16]	valid_0's binary_logloss: 0.346219
[17]	valid_0's binary_logloss: 0.341698
[18]	valid_0's binary_logloss: 0.337433
[19]	valid_0's binary_logloss: 0.333402
[20]	valid_0's binary_logloss: 0.329584
[21]	valid_0's binary_logloss: 0.325977
[22]	valid_0's binary_logloss: 0.322563
[23]	valid_0's binary_logloss: 0.31932
[24]	valid_0's binary_loglos

[203]	valid_0's binary_logloss: 0.251148
[204]	valid_0's binary_logloss: 0.25114
[205]	valid_0's binary_logloss: 0.25114
[206]	valid_0's binary_logloss: 0.251136
[207]	valid_0's binary_logloss: 0.251135
[208]	valid_0's binary_logloss: 0.251131
[209]	valid_0's binary_logloss: 0.251125
[210]	valid_0's binary_logloss: 0.251122
[211]	valid_0's binary_logloss: 0.251121
[212]	valid_0's binary_logloss: 0.251114
[213]	valid_0's binary_logloss: 0.251111
[214]	valid_0's binary_logloss: 0.251109
[215]	valid_0's binary_logloss: 0.251108
[216]	valid_0's binary_logloss: 0.251105
[217]	valid_0's binary_logloss: 0.251105
[218]	valid_0's binary_logloss: 0.251095
[219]	valid_0's binary_logloss: 0.251094
[220]	valid_0's binary_logloss: 0.251089
[221]	valid_0's binary_logloss: 0.251094
[222]	valid_0's binary_logloss: 0.25109
[223]	valid_0's binary_logloss: 0.251088
[224]	valid_0's binary_logloss: 0.25109
[225]	valid_0's binary_logloss: 0.251087
[226]	valid_0's binary_logloss: 0.251082
[227]	valid_0's bina

[405]	valid_0's binary_logloss: 0.251051
[406]	valid_0's binary_logloss: 0.251052
[407]	valid_0's binary_logloss: 0.251051
[408]	valid_0's binary_logloss: 0.251058
[409]	valid_0's binary_logloss: 0.25106
[410]	valid_0's binary_logloss: 0.25106
[411]	valid_0's binary_logloss: 0.251059
[412]	valid_0's binary_logloss: 0.251064
[413]	valid_0's binary_logloss: 0.251063
[414]	valid_0's binary_logloss: 0.251062
[415]	valid_0's binary_logloss: 0.25106
[416]	valid_0's binary_logloss: 0.25106
[417]	valid_0's binary_logloss: 0.251062
[418]	valid_0's binary_logloss: 0.251062
[419]	valid_0's binary_logloss: 0.251065
[420]	valid_0's binary_logloss: 0.251067
[421]	valid_0's binary_logloss: 0.251066
[422]	valid_0's binary_logloss: 0.25107
[423]	valid_0's binary_logloss: 0.251069
[424]	valid_0's binary_logloss: 0.25107
[425]	valid_0's binary_logloss: 0.251064
[426]	valid_0's binary_logloss: 0.251064
[427]	valid_0's binary_logloss: 0.251066
[428]	valid_0's binary_logloss: 0.251068
[429]	valid_0's binary

[105]	valid_0's binary_logloss: 0.253666
[106]	valid_0's binary_logloss: 0.25359
[107]	valid_0's binary_logloss: 0.253515
[108]	valid_0's binary_logloss: 0.253444
[109]	valid_0's binary_logloss: 0.253375
[110]	valid_0's binary_logloss: 0.253306
[111]	valid_0's binary_logloss: 0.25325
[112]	valid_0's binary_logloss: 0.253184
[113]	valid_0's binary_logloss: 0.253126
[114]	valid_0's binary_logloss: 0.253073
[115]	valid_0's binary_logloss: 0.253009
[116]	valid_0's binary_logloss: 0.252962
[117]	valid_0's binary_logloss: 0.252912
[118]	valid_0's binary_logloss: 0.252856
[119]	valid_0's binary_logloss: 0.252811
[120]	valid_0's binary_logloss: 0.252769
[121]	valid_0's binary_logloss: 0.252732
[122]	valid_0's binary_logloss: 0.252691
[123]	valid_0's binary_logloss: 0.252654
[124]	valid_0's binary_logloss: 0.252614
[125]	valid_0's binary_logloss: 0.252571
[126]	valid_0's binary_logloss: 0.252532
[127]	valid_0's binary_logloss: 0.252489
[128]	valid_0's binary_logloss: 0.252462
[129]	valid_0's bi

[306]	valid_0's binary_logloss: 0.251691
[307]	valid_0's binary_logloss: 0.25169
[308]	valid_0's binary_logloss: 0.251695
[309]	valid_0's binary_logloss: 0.251697
[310]	valid_0's binary_logloss: 0.251699
[311]	valid_0's binary_logloss: 0.251703
[312]	valid_0's binary_logloss: 0.251689
[313]	valid_0's binary_logloss: 0.25169
[314]	valid_0's binary_logloss: 0.251688
[315]	valid_0's binary_logloss: 0.251683
[316]	valid_0's binary_logloss: 0.251684
[317]	valid_0's binary_logloss: 0.251684
[318]	valid_0's binary_logloss: 0.251684
[319]	valid_0's binary_logloss: 0.251685
[320]	valid_0's binary_logloss: 0.251679
[321]	valid_0's binary_logloss: 0.251674
[322]	valid_0's binary_logloss: 0.251677
[323]	valid_0's binary_logloss: 0.251679
[324]	valid_0's binary_logloss: 0.251679
[325]	valid_0's binary_logloss: 0.251683
[326]	valid_0's binary_logloss: 0.251683
[327]	valid_0's binary_logloss: 0.251682
[328]	valid_0's binary_logloss: 0.251682
[329]	valid_0's binary_logloss: 0.251684
[330]	valid_0's bi

...

[145]	valid_0's binary_logloss: 0.254975
[146]	valid_0's binary_logloss: 0.254967
[147]	valid_0's binary_logloss: 0.254953
[148]	valid_0's binary_logloss: 0.254941
[149]	valid_0's binary_logloss: 0.254925
[150]	valid_0's binary_logloss: 0.254921
[151]	valid_0's binary_logloss: 0.254905
[152]	valid_0's binary_logloss: 0.254896
[153]	valid_0's binary_logloss: 0.25488
[154]	valid_0's binary_logloss: 0.254862
[155]	valid_0's binary_logloss: 0.25485
[156]	valid_0's binary_logloss: 0.254845
[157]	valid_0's binary_logloss: 0.25483
[158]	valid_0's binary_logloss: 0.254818
[159]	valid_0's binary_logloss: 0.254813
[160]	valid_0's binary_logloss: 0.254794
[161]	valid_0's binary_logloss: 0.254788
[162]	valid_0's binary_logloss: 0.25478
[163]	valid_0's binary_logloss: 0.25477
[164]	valid_0's binary_logloss: 0.254756
[165]	valid_0's binary_logloss: 0.25475
[166]	valid_0's binary_logloss: 0.254745
[167]	valid_0's binary_logloss: 0.254736
[168]	valid_0's binary_logloss: 0.254736
[169]	valid_0's binary

[347]	valid_0's binary_logloss: 0.254513
[348]	valid_0's binary_logloss: 0.254516
[349]	valid_0's binary_logloss: 0.254512
[350]	valid_0's binary_logloss: 0.254511
[351]	valid_0's binary_logloss: 0.254506
[352]	valid_0's binary_logloss: 0.254506
[353]	valid_0's binary_logloss: 0.254508
[354]	valid_0's binary_logloss: 0.254505
[355]	valid_0's binary_logloss: 0.254506
[356]	valid_0's binary_logloss: 0.254504
[357]	valid_0's binary_logloss: 0.254503
[358]	valid_0's binary_logloss: 0.254503
[359]	valid_0's binary_logloss: 0.254502
[360]	valid_0's binary_logloss: 0.254502
[361]	valid_0's binary_logloss: 0.254506
[362]	valid_0's binary_logloss: 0.254499
[363]	valid_0's binary_logloss: 0.2545
[364]	valid_0's binary_logloss: 0.254501
[365]	valid_0's binary_logloss: 0.254504
[366]	valid_0's binary_logloss: 0.254502
[367]	valid_0's binary_logloss: 0.2545
[368]	valid_0's binary_logloss: 0.2545
[369]	valid_0's binary_logloss: 0.254497
[370]	valid_0's binary_logloss: 0.254501
[371]	valid_0's binary

[549]	valid_0's binary_logloss: 0.254593
[550]	valid_0's binary_logloss: 0.254595
[551]	valid_0's binary_logloss: 0.254599
[552]	valid_0's binary_logloss: 0.254594
[553]	valid_0's binary_logloss: 0.254594
[554]	valid_0's binary_logloss: 0.254598
[555]	valid_0's binary_logloss: 0.254598
[556]	valid_0's binary_logloss: 0.254602
[557]	valid_0's binary_logloss: 0.254606
[558]	valid_0's binary_logloss: 0.254605
[559]	valid_0's binary_logloss: 0.254606
[560]	valid_0's binary_logloss: 0.254605
[561]	valid_0's binary_logloss: 0.254609
[562]	valid_0's binary_logloss: 0.254608
[563]	valid_0's binary_logloss: 0.25461
[564]	valid_0's binary_logloss: 0.25461
[565]	valid_0's binary_logloss: 0.254608
[566]	valid_0's binary_logloss: 0.254607
[567]	valid_0's binary_logloss: 0.254609
[568]	valid_0's binary_logloss: 0.254612
[569]	valid_0's binary_logloss: 0.254611
Early stopping, best iteration is:
[369]	valid_0's binary_logloss: 0.254497
[1]	valid_0's binary_logloss: 0.465717
Training until validation 

[181]	valid_0's binary_logloss: 0.251149
[182]	valid_0's binary_logloss: 0.251141
[183]	valid_0's binary_logloss: 0.251134
[184]	valid_0's binary_logloss: 0.251138
[185]	valid_0's binary_logloss: 0.251131
[186]	valid_0's binary_logloss: 0.251135
[187]	valid_0's binary_logloss: 0.251136
[188]	valid_0's binary_logloss: 0.251132
[189]	valid_0's binary_logloss: 0.251133
[190]	valid_0's binary_logloss: 0.251127
[191]	valid_0's binary_logloss: 0.251129
[192]	valid_0's binary_logloss: 0.251124
[193]	valid_0's binary_logloss: 0.251123
[194]	valid_0's binary_logloss: 0.251121
[195]	valid_0's binary_logloss: 0.251122
[196]	valid_0's binary_logloss: 0.25112
[197]	valid_0's binary_logloss: 0.25112
[198]	valid_0's binary_logloss: 0.251116
[199]	valid_0's binary_logloss: 0.251105
[200]	valid_0's binary_logloss: 0.251109
[201]	valid_0's binary_logloss: 0.251111
[202]	valid_0's binary_logloss: 0.251116
[203]	valid_0's binary_logloss: 0.251114
[204]	valid_0's binary_logloss: 0.251116
[205]	valid_0's bi

[384]	valid_0's binary_logloss: 0.251186
[385]	valid_0's binary_logloss: 0.251183
[386]	valid_0's binary_logloss: 0.251179
[387]	valid_0's binary_logloss: 0.251177
[388]	valid_0's binary_logloss: 0.251177
[389]	valid_0's binary_logloss: 0.251179
[390]	valid_0's binary_logloss: 0.25118
[391]	valid_0's binary_logloss: 0.251181
[392]	valid_0's binary_logloss: 0.251188
[393]	valid_0's binary_logloss: 0.251189
[394]	valid_0's binary_logloss: 0.25119
[395]	valid_0's binary_logloss: 0.251189
[396]	valid_0's binary_logloss: 0.251191
[397]	valid_0's binary_logloss: 0.251192
[398]	valid_0's binary_logloss: 0.251193
[399]	valid_0's binary_logloss: 0.251196
[400]	valid_0's binary_logloss: 0.251197
[401]	valid_0's binary_logloss: 0.251197
[402]	valid_0's binary_logloss: 0.251198
[403]	valid_0's binary_logloss: 0.251199
[404]	valid_0's binary_logloss: 0.2512
[405]	valid_0's binary_logloss: 0.251201
[406]	valid_0's binary_logloss: 0.251206
[407]	valid_0's binary_logloss: 0.251206
[408]	valid_0's bina

In [37]:
avg = (pred_test[:,1])/5
avg

array([0.78152312, 0.64326449, 0.11878676, ..., 0.03590093, 0.00864467,
       0.05048876])

In [41]:
#preparing the predictions make into the sample file for submission

#preds1 = xg.predict(test_enc)
sample1 = pd.read_csv('sample_submission.csv')
test = pd.read_csv('Test.csv')
sample1.user_id = test.user_id
sample1.CHURN = avg
sample1.to_csv('LGB-5-FOLD-61-features.csv', index = False)

In [44]:
avg = (pred_test[:,1])/5
avg

array([0.78101235, 0.64843751, 0.12482533, ..., 0.03542726, 0.0082081 ,
       0.05046392])

In [46]:
#preparing the predictions make into the sample file for submission

#preds1 = xg.predict(test_enc)
sample1 = pd.read_csv('sample_submission.csv')
test = pd.read_csv('Test.csv')
sample1.user_id = test.user_id
sample1.CHURN = avg
sample1.to_csv('LGB-maxdepth-10-FOLD-61-features.csv', index = False)